In [ ]:
# import sys
# !{sys.executable} -m pip install pandas

In [1]:
import sys
import os
workPath = os.path.abspath('..')
if not workPath in sys.path:
    sys.path.append(workPath)

In [2]:
import os
import shutil
import tarfile
import h5py
import pandas as pd
import json
from everest import _specialnames

# import os
# inputPath = os.path.join(*['..', 'remote', 'transfer', 'VAULT', 'data', 'isobench', 'isobenchJune', 'res32'])
# projName = 'isobench32'
# targetFilename = 'zerodData.csv'
# inputFilename = 'inputs.txt'

def legacy(
        inputPath,
        projName,
        targetFilename,
        inputFilename
        ):
    outputPath = os.path.join(*['..', 'out', 'analysis', projName])
    h5filename = os.path.join(outputPath, projName + '.h5')

    def clean_df_colnames(df):
        df.columns = df.columns.str.strip('#')
        df.columns = df.columns.str.strip().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
        df.rename(columns = {
                'step': _specialnames.COUNTS_FLAG
                },
            inplace = True
            )

    pemods = [
        os.path.splitext(os.path.splitext(filename)[0])[0] \
            for filename in os.listdir(inputPath) \
                if filename[:6] == 'pemod_'
        ]
    if os.path.exists(outputPath):
        shutil.rmtree(outputPath)
    os.makedirs(outputPath, exist_ok = True)
    with h5py.File(h5filename, 'w') as myfile:
        for pemod in pemods:
            # create group
            group = myfile.create_group(pemod)
            # get files from archive
            filename = os.path.join(inputPath, pemod + '.tar.gz')
            with tarfile.open(filename) as file:
                file.extract(targetFilename, outputPath)
                file.extract(inputFilename, outputPath)
            # read inputs and set as attrs
            inputFilepath = os.path.join(outputPath, inputFilename)
            with open(inputFilepath) as inputFile:
                inputDict = json.load(inputFile)
            if inputDict['params']['f'] == 'max':
                inputDict['params']['f'] = 1.
            for key, paramDict in inputDict.items():
                for key, val in paramDict.items():
                    try:
                        group.attrs[key] = val
                    except:
                        group.attrs[key] = str(val)
            os.remove(inputFilepath)
            # read data and set as datasets
            targetFilepath = os.path.join(outputPath, targetFilename)
            df = pd.read_csv(targetFilepath)
            clean_df_colnames(df)
            for key in df.keys():
                if key == _specialnames.COUNTS_FLAG:
                    dtype = 'i8'
                else:
                    dtype = df[key].dtype
                group.create_dataset(
                    name = key,
                    data = df[key],
                    chunks = True,
                    dtype = dtype
                    )
            os.remove(targetFilepath)

In [3]:
# import os
# inputPath = os.path.join(*['..', 'remote', 'transfer', 'VAULT', 'data', 'isobench', 'isobenchJune', 'res32'])
# projName = 'isobench32'
# targetFilename = 'zerodData.csv'
# inputFilename = 'inputs.txt'

In [4]:
import os
inputPath = os.path.join(*['..', 'remote', 'transfer', 'VAULT', 'data', 'arrbench', 'res16'])
projName = 'arrbench16'
targetFilename = 'zerodData.csv'
inputFilename = 'inputs.txt'

In [5]:
legacy(inputPath, projName, targetFilename, inputFilename)

KeyError: "filename 'zerodData.csv' not found"

In [ ]:
myreader = Reader(h5filename)

In [ ]:
myscope1 = myreader[
    Fetch('Ra') > 1e6,
    Fetch('Ra') < 4e6,
    Fetch('modeltime') > 0.01
    ]

In [ ]:
myscope3 = myscope1 * myscope2
myscope3

In [ ]:
# inFile = h5py.File(h5filename, 'r')
# outFile = h5py.File(outFilename, 'w')
# inFile.close()

In [ ]:
# inputFrame = dataframe.read_hdf(h5filename, '/*')

In [ ]:
# res = 32
# maxtime = .3

# rowName = 'Ra'
# rows = sorted(list(set(inputFrame.Ra.loc[inputFrame.Ra <= 1e5])))
# cols = ['Nu', 'VRMS', 'avTemp']

# fig, axs = plt.subplots(len(rows), len(cols), sharex=True, figsize = (12, 12))

# for row, val in enumerate(rows):
#     indices = inputFrame.loc[inputFrame[rowName] == val].loc[inputFrame['res'] == res].index
#     for index in indices:
#         frame = outputFrame.loc[outputFrame['modeltime'] < maxtime].loc[index]
#         for col, val in enumerate(cols):
#             ignoreme = axs[row][col].plot(frame.modeltime, frame[val])

# for index, val in enumerate(cols):
#     axs[0][index].set_title(val)
# for index, val in enumerate(rows):
#     axs[index][0].set_ylabel(val)
# axs[len(rows) - 1][int(len(cols) / 2)].set_xlabel('Dimensionless time')

# legendtext = ['f = ' + str(round(val, 2)) for val in list(inputFrame.loc[indices].f)]
# legend = plt.figlegend(
#     legendtext,
#     loc='right',
#     shadow = True,
# #     mode = 'expand',
#     )
# fig.subplots_adjust(hspace=0)
# # fig.suptitle('Isoviscous suite: res = ' + str(res))
# fig.show()
# plt.savefig('isobenchFullchunk1.png', dpi=300)